In [2]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from models.header import get_head_from_outputs

In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Conv2D, MaxPool2D, Activation, SeparableConv2D
from tensorflow.keras.regularizers import l2

class HeadWrapper(Layer):
    """Merging all feature maps for detections.
    inputs:
        conv4_3 = (batch_size, (layer_shape x aspect_ratios), last_dimension)
            ssd300 conv4_3 shape => (38 x 38 x 4) = 5776
        conv7 = (batch_size, (layer_shape x aspect_ratios), last_dimension)
            ssd300 conv7 shape => (19 x 19 x 6) = 2166
        conv8_2 = (batch_size, (layer_shape x aspect_ratios), last_dimension)
            ssd300 conv8_2 shape => (10 x 10 x 6) = 600
        conv9_2 = (batch_size, (layer_shape x aspect_ratios), last_dimension)
            ssd300 conv9_2 shape => (5 x 5 x 6) = 150
        conv10_2 = (batch_size, (layer_shape x aspect_ratios), last_dimension)
            ssd300 conv10_2 shape => (3 x 3 x 4) = 36
        conv11_2 = (batch_size, (layer_shape x aspect_ratios), last_dimension)
            ssd300 conv11_2 shape => (1 x 1 x 4) = 4
                                           Total = 8732 default box

    outputs:
        merged_head = (batch_size, total_prior_boxes, last_dimension)
    """

    def __init__(self, last_dimension, **kwargs):
        super(HeadWrapper, self).__init__(**kwargs)
        self.last_dimension = last_dimension

    def get_config(self):
        config = super(HeadWrapper, self).get_config()
        config.update({"last_dimension": self.last_dimension})
        return config

    def call(self, inputs):
        last_dimension = self.last_dimension
        batch_size = tf.shape(inputs[0])[0]
        outputs = []
        for conv_layer in inputs:
            outputs.append(tf.reshape(conv_layer, (batch_size, -1, last_dimension)))
        #
        return tf.concat(outputs, axis=1)

def get_head_from_outputs(hyper_params, outputs):
    """Generating ssd bbox delta and label heads.
    inputs:
        hyper_params = dictionary
        outputs = list of ssd layers output to be used for prediction

    outputs:
        pred_deltas = merged outputs for bbox delta head
        pred_labels = merged outputs for bbox label head
    """
    label_reg_factor = 5e-3
    box_reg_factor = 1e-3
    total_labels = hyper_params["total_labels"]
    # +1 for ratio 1
    len_aspect_ratios = [len(x) + 1 for x in hyper_params["aspect_ratios"]]
    labels_head = []
    boxes_head = []
    for i, output in enumerate(outputs):
        aspect_ratio = len_aspect_ratios[i]
#         labels_head.append(Conv2D(aspect_ratio * total_labels, (3, 3), padding="same", name="{}_conv_label_output".format(i+1))(output))
#         boxes_head.append(Conv2D(aspect_ratio * 4, (3, 3), padding="same", name="{}_conv_boxes_output".format(i+1))(output))
        
        labels_head.append(SeparableConv2D(filters = aspect_ratio * total_labels, kernel_size = (3, 3), padding = "same", name = f"{i+1}_separable_conv_label_output", depthwise_regularizer = l2(label_reg_factor), pointwise_regularizer = l2(label_reg_factor))(output))
        boxes_head.append(SeparableConv2D(filters = aspect_ratio * 4, kernel_size = (3, 3), padding = "same", name = f"{i+1}_separable_conv_boxes_output", depthwise_regularizer = l2(box_reg_factor), pointwise_regularizer = l2(box_reg_factor))(output))
    
    pred_labels = HeadWrapper(total_labels, name="labels_head")(labels_head)
    pred_labels = Activation("softmax", name="conf")(pred_labels)
    
    pred_deltas = HeadWrapper(4, name="loc")(boxes_head)
    return pred_deltas, pred_labels

In [31]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from models.header import get_head_from_outputs

def get_model(hyper_params = None):
    """Generating ssd model for hyper params.
    inputs:
        hyper_params = dictionary

    outputs:
        ssd_model = tf.keras.model
    """
    reg_factor = 1e-3
    img_size = 300
    base_model = MobileNetV2(include_top=False, input_shape=(img_size, img_size, 3))
    input = base_model.input
    first_conv = base_model.get_layer("block_13_expand_relu").output
    second_conv = base_model.output
    #
    ############################ Extra Feature Layers Start ############################
    extra1_1 = Conv2D(256, (1, 1), strides=(1, 1), padding="valid", activation="relu", name="extra1_1", kernel_regularizer = l2(reg_factor))(second_conv)
    extra1_2 = Conv2D(512, (3, 3), strides=(2, 2), padding="same", activation="relu", name="extra1_2", kernel_regularizer = l2(reg_factor))(extra1_1)
    #
    extra2_1 = Conv2D(128, (1, 1), strides=(1, 1), padding="valid", activation="relu", name="extra2_1", kernel_regularizer = l2(reg_factor))(extra1_2)
    extra2_2 = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu", name="extra2_2", kernel_regularizer = l2(reg_factor))(extra2_1)
    #
    extra3_1 = Conv2D(128, (1, 1), strides=(1, 1), padding="valid", activation="relu", name="extra3_1", kernel_regularizer = l2(reg_factor))(extra2_2)
    extra3_2 = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu", name="extra3_2", kernel_regularizer = l2(reg_factor))(extra3_1)
    #
    extra4_1 = Conv2D(128, (1, 1), strides=(1, 1), padding="valid", activation="relu", name="extra4_1", kernel_regularizer = l2(reg_factor))(extra3_2)
    extra4_2 = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu", name="extra4_2", kernel_regularizer = l2(reg_factor))(extra4_1)
    ############################ Extra Feature Layers End ############################
    pred_deltas, pred_labels = get_head_from_outputs(hyper_params, [first_conv, second_conv, extra1_2, extra2_2, extra3_2, extra4_2])
    return Model(inputs=input, outputs=[pred_deltas, pred_labels])

def init_model(model):
    """Initializing model with dummy data for load weights with optimizer state and also graph construction.
    inputs:
        model = tf.keras.model

    """
    model(tf.random.uniform((1, 512, 512, 3)))

In [32]:
from utils import train_utils
hyper_params = train_utils.get_hyper_params("mobilenet_v2", True)
hyper_params['total_labels'] = 59

In [33]:
ssd_model = get_model(hyper_params)

In [34]:
ssd_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 150, 150, 32  864         ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 150, 150, 32  128         ['Conv1[0][0]']                  
                                )                                                           

 block_3_expand_relu (ReLU)     (None, 75, 75, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 77, 77, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 38, 38, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 38, 38, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 19, 19, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 19, 19, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 19, 19, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 19, 19, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 19, 19, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 19, 19, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 19, 19, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 19, 19, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 10, 10, 960)  8640       ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 10, 10, 960)  3840       ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 10, 10, 960)  0          ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 10, 10, 160)  153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

                                                                                                  
 1_separable_conv_label_output   (None, 19, 19, 236)  141356     ['block_13_expand_relu[0][0]']   
 (SeparableConv2D)                                                                                
                                                                                                  
 2_separable_conv_label_output   (None, 10, 10, 354)  464994     ['out_relu[0][0]']               
 (SeparableConv2D)                                                                                
                                                                                                  
 3_separable_conv_label_output   (None, 5, 5, 354)   186210      ['extra1_2[0][0]']               
 (SeparableConv2D)                                                                                
                                                                                                  
 4_separab

In [36]:
from models import ssd_vgg16

In [ ]:
model = get_model(train_utils("vgg16"))